# Installs + Imports

In [5]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tqdm.auto import tqdm
from datetime import datetime 
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Helpers

In [21]:
class LoadDataset(Dataset):
  def __init__(self, X_df, y_df):
    self.X = torch.tensor(X_df.values, dtype=torch.float32)
    self.y = torch.tensor(y_df.values, dtype=torch.float32)
  
  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [22]:
class FeedForward(nn.Module):
  def __init__(self,
               n_layers,
               layers,
               activation=nn.ReLU()):

    super().__init__()
    self.n_layers = n_layers 
    self.layers = layers
    self.activation = activation

    dense_layers = [
        self.dense_layer(in_features=self.layers[i],
                         out_features=self.layers[i+1])
        for i in range(self.n_layers-1)]
    dense_layers.append(nn.Linear(in_features=self.layers[-2],
                                  out_features=self.layers[-1]))

    self.feed_forward = nn.Sequential(*dense_layers)

  def dense_layer(self, in_features, out_features):
    dense_layer = nn.Sequential(
      nn.Linear(in_features=in_features,
                out_features=out_features),
      self.activation,
    )
    return dense_layer
  
  def forward(self, x):
    return self.feed_forward(x)

In [23]:
def mape(y_preds, y_true):
  epsilon = 1.17e-06
  abs_diff = torch.abs(y_preds - y_true)
  abs_per_error = abs_diff / torch.clamp(torch.abs(y_true), min=epsilon)
  mape = torch.sum(abs_per_error) / y_true.numel()

  return mape 

# def loss_fn(y_preds, y):
#   loss = mape(y_preds, y) * (1 + 0.4 * torch.max(torch.tensor([0, mape(y_preds, y) - 2])))
#   return loss.requires_grad_(True)

def loss_fn(y_preds, y):
  MAPE = mape(y_preds, y)
  loss = MAPE if MAPE < 10 else torch.exp(MAPE - 10) 
  return loss.requires_grad_(True)

def train_step(model,
               dataloader, 
               optimizer, 
               device):
  
  model.train()
  loss = 0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_preds = model(X).squeeze()
    batch_loss = loss_fn(y_preds, y)
    loss += batch_loss.item()
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
  
  loss /= len(dataloader)
  return loss

def val_step(model, dataloader, device):
  model.eval()
  val_loss = 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_preds = model(X).squeeze()

      y_preds_unscaled = y_preds * scaler.max_abs_[TARGET_POS]
      y_true_unscaled  = y * scaler.max_abs_[TARGET_POS] 

      batch_loss = 100 * mape(y_preds_unscaled, y_true_unscaled) 
      val_loss += batch_loss.item()
  
  val_loss /= len(dataloader)
  return val_loss

def train(model, 
          train_dataloader,
          val_dataloader,
          optimizer,
          scheduler,
          epochs,
          patience,
          device,
          path):
  
  results = {
      "loss": [],
      "val_loss": []
  }

  for epoch in tqdm(range(epochs)):
    flag = 0
    loss = train_step(model=model,
                      dataloader=train_dataloader,
                      optimizer=optimizer,
                      device=device)

    val_loss = val_step(model=model,
                        dataloader=val_dataloader,
                        device=device)
    scheduler.step(val_loss)
    
    results['loss'].append(loss)
    results['val_loss'].append(val_loss)
    if epoch == 0:
      best_val_loss = val_loss
      best_epoch = -1
      checkpoint(model, optimizer, path)
      flag = 1
      print(f"Epoch: {epoch+1}/{epochs} | Loss: {loss:.4f} | Val loss: {val_loss:.4f} - *Checkpoint*")
    else:
      if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        checkpoint(model, optimizer, path)
        flag = 1
        print(f"Epoch: {epoch+1}/{epochs} | Loss: {loss:.4f} | Val loss: {val_loss:.4f} - *Checkpoint*")
      elif epoch - best_epoch > patience:
        print(f"\nEarly stopping applied at epoch {epoch}.")
        break
    if flag == 0:
      print(f"Epoch: {epoch+1}/{epochs} | Loss: {loss:.4f} | Val loss: {val_loss:.4f}")
  
  return results

def checkpoint(model, optimizer, filepath):
  torch.save({
    "optimizer": optimizer.state_dict(),
    "model": model.state_dict()
  }, filepath)

# Preprocessing

In [17]:
TARGET = "TOTAL_CONS"

# keep 1 year for testing
START_TEST_DATE = pd.to_datetime('2018-01-01')
END_TEST_DATE = START_TEST_DATE + pd.DateOffset(years=1)

END_VAL_DATE = START_TEST_DATE - pd.to_timedelta(1, 'h')
START_VAL_DATE = pd.to_datetime('2017-01-01')

START_TRAIN_DATE = pd.to_datetime('2010-01-01')
END_TRAIN_DATE = START_VAL_DATE - pd.to_timedelta(1, 'h')

DAYS_BACK_TO_SKIP = 0

START_STEP_FORWARD = 24 * 10
LAST_STEP_FORWARD = START_STEP_FORWARD + 24
STEPS_BACKWARD_START = 24 * DAYS_BACK_TO_SKIP

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
df = pd.read_csv("/content/FINAL_DATASET_2.csv")
df.set_index(pd.to_datetime(df["Timestamp"]), inplace=True)
df.drop("Timestamp", axis=1, inplace=True)
df

,TOTAL_CONS,Weekend,Holiday,temp,humidity,hour,weekday,dayofyear
Timestamp,,,,,,,,
2010-10-01 00:00:00,4390.054,0,0,16.12,79.0,0,4,274
2010-10-01 01:00:00,4046.071,0,0,15.19,77.0,1,4,274
2010-10-01 02:00:00,3885.451,0,0,14.65,82.0,2,4,274
2010-10-01 03:00:00,3808.100,0,0,14.03,71.0,3,4,274
2010-10-01 04:00:00,3782.623,0,0,13.29,77.0,4,4,274
...,...,...,...,...,...,...,...,...
2020-11-22 19:00:00,4281.942,1,0,10.74,54.0,19,6,327
2020-11-22 20:00:00,4091.488,1,0,10.15,51.0,20,6,327
2020-11-22 21:00:00,3738.827,1,0,9.81,51.0,21,6,327


In [19]:
TARGET_POS = np.where(df.columns == TARGET)[0][0]

train_val_df = df[(df.index >= START_TRAIN_DATE) & (df.index <= END_VAL_DATE)]
test_df = df[(df.index >= START_TEST_DATE)]

scaler = MaxAbsScaler()    # MinMaxScaler()
train_val_scaled = scaler.fit_transform(train_val_df)
train_val_df_scaled = pd.DataFrame(train_val_scaled,
                                   columns=train_val_df.columns,
                                   index=train_val_df.index)
test_scaled = scaler.transform(test_df)
test_df_scaled = pd.DataFrame(test_scaled,
                              columns=test_df.columns,
                              index=test_df.index)

scaled_df = pd.concat([train_val_df_scaled, test_df_scaled], axis=0)
scaled_df.drop(['humidity', 'hour', 'dayofyear'], axis=1, inplace=True)
scaled_df

,TOTAL_CONS,Weekend,Holiday,temp,weekday
Timestamp,,,,,
2010-10-01 00:00:00,0.443017,0.0,0.0,0.419682,0.666667
2010-10-01 01:00:00,0.408305,0.0,0.0,0.395470,0.666667
2010-10-01 02:00:00,0.392096,0.0,0.0,0.381411,0.666667
2010-10-01 03:00:00,0.384290,0.0,0.0,0.365269,0.666667
2010-10-01 04:00:00,0.381719,0.0,0.0,0.346004,0.666667
...,...,...,...,...,...
2020-11-22 19:00:00,0.432107,1.0,0.0,0.279615,1.000000
2020-11-22 20:00:00,0.412888,1.0,0.0,0.264254,1.000000
2020-11-22 21:00:00,0.377299,1.0,0.0,0.255402,1.000000


# Training + Inference (one step at a time)

In [ ]:
mape_list = list()
# for step_forward in np.arange(START_STEP_FORWARD, LAST_STEP_FORWARD + 1, step=1):
for step_forward in range(1):

  scaled_df_copy = scaled_df.copy()  # pd.DataFrame()

  # *** shift Weather + Time data ***
  for col in scaled_df.drop(TARGET, axis=1).columns:
    scaled_df_copy[col + f"_(t+{step_forward})"] = scaled_df[col].shift(-step_forward)
    scaled_df_copy.drop(col, axis=1, inplace=True)

  # ---------- REFRAMING -----------------
  # *** shift Target ***
  scaled_df_copy[TARGET + f"_(t+{step_forward})"] = scaled_df[TARGET].shift(-step_forward)

  # *** shift Backsteps *** 
  for day_back in np.arange(DAYS_BACK_TO_SKIP + 1, DAYS_BACK_TO_SKIP + 10, step=1):
    step_back = 24 * day_back - step_forward
    scaled_df_copy[TARGET + f"_(t-{step_back})"] = scaled_df[TARGET].shift(step_back)
      
  refr_df = scaled_df_copy.drop(TARGET, axis=1).copy()
  refr_df.dropna(inplace=True)

  # print()
  # print(np.abs(refr_df.corr())[TARGET + f"_(t+{step_forward})"])
  # print()
  # print(refr_df.columns)

  # split to train, validation and test sets
  train_df_refr = refr_df[(refr_df.index >= START_TRAIN_DATE) & (refr_df.index <= END_TRAIN_DATE)]
  val_df_refr = refr_df[(refr_df.index >= START_VAL_DATE) & (refr_df.index <= END_VAL_DATE)]
  test_df_refr = refr_df[(refr_df.index >= START_TEST_DATE) & (refr_df.index <= END_TEST_DATE)]

  # shuffle train set
  train_df_refr = shuffle(train_df_refr)

  # split to features and targets
  X_train_df = train_df_refr.drop(TARGET + f"_(t+{step_forward})", axis=1)
  y_train_df = train_df_refr[TARGET + f"_(t+{step_forward})"]

  X_val_df = val_df_refr.drop(TARGET + f"_(t+{step_forward})", axis=1)
  y_val_df = val_df_refr[TARGET + f"_(t+{step_forward})"]

  X_test_df = test_df_refr.drop(TARGET + f"_(t+{step_forward})", axis=1)
  y_test_df = test_df_refr[TARGET + f"_(t+{step_forward})"]

  # *** DATALOADERS ***
  train_dataset = LoadDataset(X_df=X_train_df,
                              y_df=y_train_df)
  train_dataloader = DataLoader(dataset=train_dataset, 
                                batch_size=BATCH_SIZE,
                                shuffle=True)

  val_dataset = LoadDataset(X_df=X_val_df,
                            y_df=y_val_df)
  val_dataloader = DataLoader(dataset=val_dataset, 
                              batch_size=BATCH_SIZE,
                              shuffle=False)
  
  test_dataset = LoadDataset(X_df=X_test_df,
                             y_df=y_test_df)
  test_dataloader = DataLoader(dataset=test_dataset, 
                               batch_size=BATCH_SIZE,
                               shuffle=False)

  # Defince the model
  N_NEURONS = 32
  LAYERS = [X_train_df.shape[1], 300, 100, 1]
  N_LAYERS = len(LAYERS) - 1
  model = FeedForward(n_layers=N_LAYERS,
                    layers=LAYERS,
                    activation=nn.ReLU()).to(device)
  !rm -rf "model.pth"
  optimizer = torch.optim.Adam(params=model.parameters(),
                               lr=1e-3,
                               weight_decay=0)
  # optimizer = t.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
  scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.33, patience=15, verbose=True)

  # Training Feed-Forward ...
  model_results = train(model=model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        scheduler=scheduler,
                        epochs=EPOCHS,
                        patience=PATIENCE,
                        device=device,
                        path=PATH)
  
  # Inference
  model.eval()
  with torch.inference_mode():
    for batch, (X_test, y_test) in enumerate(test_dataloader):
      X_test, y_test= X_test.to(device), y_test.to(device)
      test_batch_preds = model(X_test).squeeze()
      if batch == 0:
        y_step_preds_scaled = test_batch_preds
        y_step_test_scaled = y_test
      else:
        y_step_preds_scaled = torch.cat((y_step_preds_scaled, test_batch_preds), dim=0)
        y_step_test_scaled = torch.cat((y_step_test_scaled, y_test), dim=0)

  y_step_preds = y_step_preds_scaled.to('cpu').numpy() * scaler.max_abs_[TARGET_POS]
  y_step_test = y_step_test_scaled.to('cpu').numpy() * scaler.max_abs_[TARGET_POS] 

  step_results = pd.DataFrame(
      {
        "real": y_step_test,
        "predictions": y_step_preds
      },
      index=y_test_df.index + pd.to_timedelta(step_forward, 'h')
  )

  step_results['abs_error'] = np.abs(step_results["real"] - step_results["predictions"])
  step_results['ape'] = np.where(step_results["real"] == 0, np.NaN, 100 * step_results['abs_error'] / step_results["real"])
  step_mape = step_results['ape'].mean()
  mape_list.append(step_mape)

  print(f"step {step_forward} -> MAPE = {step_mape}%")
MAPE = np.mean(np.array(mape_list))
print(f"\nOverall MAPE = {MAPE}%")

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 1/2000 | Loss: 0.4034 | Val loss: 14.8940 - *Checkpoint*
Epoch: 2/2000 | Loss: 0.1136 | Val loss: 11.8796 - *Checkpoint*
Epoch: 3/2000 | Loss: 0.0845 | Val loss: 8.8849 - *Checkpoint*
Epoch: 4/2000 | Loss: 0.0737 | Val loss: 8.0902 - *Checkpoint*
Epoch: 5/2000 | Loss: 0.0698 | Val loss: 7.6466 - *Checkpoint*
Epoch: 6/2000 | Loss: 0.0681 | Val loss: 7.5322 - *Checkpoint*
Epoch: 7/2000 | Loss: 0.0672 | Val loss: 7.6659
Epoch: 8/2000 | Loss: 0.0668 | Val loss: 7.4491 - *Checkpoint*
Epoch: 9/2000 | Loss: 0.0657 | Val loss: 7.2932 - *Checkpoint*
Epoch: 10/2000 | Loss: 0.0648 | Val loss: 7.2889 - *Checkpoint*
Epoch: 11/2000 | Loss: 0.0643 | Val loss: 7.3525
Epoch: 12/2000 | Loss: 0.0634 | Val loss: 7.1298 - *Checkpoint*
Epoch: 13/2000 | Loss: 0.0628 | Val loss: 7.0305 - *Checkpoint*
Epoch: 14/2000 | Loss: 0.0621 | Val loss: 7.1869
Epoch: 15/2000 | Loss: 0.0617 | Val loss: 6.9078 - *Checkpoint*
Epoch: 16/2000 | Loss: 0.0610 | Val loss: 7.0095
Epoch: 17/2000 | Loss: 0.0605 | Val loss: 6

In [ ]:
df[df.index == '2018-01-01 00:00:00']

,TOTAL_CONS,Weekend,Holiday,temp,humidity,hour,weekday,dayofyear
Timestamp,,,,,,,,
2018-01-01,4681.077,0,1,3.12,93.0,0,0,1


In [ ]:
model.state_dict

<bound method Module.state_dict of FeedForward(
  (activation): ReLU()
  (feed_forward): Sequential(
    (0): Sequential(
      (0): Linear(in_features=17, out_features=256, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
    )
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)>

# Training + Inference (all steps together)

In [24]:
! rm -rf "model.pth"
EPOCHS = 1000
PATIENCE = 31
PATH = "model.pth"
BATCH_SIZE = 1024

mape_list = list()
scaled_df_copy = scaled_df.copy()

# *** shift future Time data ***
for col in ['Weekend', 'Holiday', 'weekday']:
  scaled_df_copy[col + f"_(t+{START_STEP_FORWARD})"] = scaled_df[col].shift(-START_STEP_FORWARD)
  scaled_df_copy.drop(col, axis=1, inplace=True)

# *** shift future Weather data ***
for col in ['temp']:
  for i in range(START_STEP_FORWARD + 1, LAST_STEP_FORWARD + 1):
    scaled_df_copy[col + f"_(t+{i})"] = scaled_df[col].shift(-i)

# ---------- REFRAMING -----------------
# *** shift Backsteps *** 
for col in ['temp', TARGET]:
  for day_back in [0, 1, 6]:
    for i in range(24):
      step_back = day_back * 24 + i
      scaled_df_copy[col + f"_(t-{step_back})"] = scaled_df[col].shift(step_back)

# *** shift Target ***
for step_forward in range(START_STEP_FORWARD + 1, LAST_STEP_FORWARD + 1):
  scaled_df_copy[TARGET + f"_(t+{step_forward})"] = scaled_df[TARGET].shift(-step_forward)
    
refr_df = scaled_df_copy.drop(['temp', TARGET], axis=1)
refr_df.dropna(inplace=True)

# print()
# print(np.abs(refr_df.corr())[TARGET + f"_(t+{1})"])
# print()
# print(refr_df.columns)

# split to train, validation and test sets
train_df_refr = refr_df[(refr_df.index >= START_TRAIN_DATE) & (refr_df.index <= END_TRAIN_DATE)]
val_df_refr = refr_df[(refr_df.index >= START_VAL_DATE) & (refr_df.index <= END_VAL_DATE)]
test_df_refr = refr_df[(refr_df.index >= START_TEST_DATE) & (refr_df.index <= END_TEST_DATE)]

# shuffle train set
train_df_refr = shuffle(train_df_refr)

# split to features and targets
X_train_df = train_df_refr.iloc[:, :-24]
y_train_df = train_df_refr.iloc[:, -24:]

X_val_df = val_df_refr.iloc[:, :-24]
y_val_df = val_df_refr.iloc[:, -24:]

X_test_df = test_df_refr.iloc[:, :-24]
y_test_df = test_df_refr.iloc[:, -24:]

# *** DATALOADERS ***
train_dataset = LoadDataset(X_df=X_train_df,
                            y_df=y_train_df)
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=BATCH_SIZE,
                              shuffle=True)

val_dataset = LoadDataset(X_df=X_val_df,
                          y_df=y_val_df)
val_dataloader = DataLoader(dataset=val_dataset, 
                            batch_size=BATCH_SIZE,
                            shuffle=False)

test_dataset = LoadDataset(X_df=X_test_df,
                            y_df=y_test_df)
test_dataloader = DataLoader(dataset=test_dataset, 
                              batch_size=BATCH_SIZE,
                              shuffle=False)

# Defince the model
LAYERS = [X_train_df.shape[1], 300, 100, 24]
N_LAYERS = len(LAYERS) - 1
model = FeedForward(n_layers=N_LAYERS,
                  layers=LAYERS,
                  activation=nn.ReLU()).to(device)
!rm -rf "model.pth"
optimizer = torch.optim.Adam(params=model.parameters(),
                              lr=1e-3,
                              weight_decay=0)
# optimizer = t.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.33, patience=15, verbose=True)

# Training Feed-Forward ...
model_results = train(model=model, 
                      train_dataloader=train_dataloader,
                      val_dataloader=val_dataloader,
                      optimizer=optimizer,
                      scheduler=scheduler,
                      epochs=EPOCHS,
                      patience=PATIENCE,
                      device=device,
                      path=PATH)

# Inference
model.eval()
with torch.inference_mode():
  for batch, (X_test, y_test) in enumerate(test_dataloader):
    X_test, y_test= X_test.to(device), y_test.to(device)
    test_batch_preds = model(X_test).squeeze()
    if batch == 0:
      y_preds_scaled = test_batch_preds
      y_test_scaled = y_test
    else:
      y_preds_scaled = torch.cat((y_preds_scaled, test_batch_preds), dim=0)
      y_test_scaled = torch.cat((y_test_scaled, y_test), dim=0)

y_preds = y_preds_scaled.to('cpu').numpy() * scaler.max_abs_[TARGET_POS]
y_test = y_test_scaled.to('cpu').numpy() * scaler.max_abs_[TARGET_POS] 

mape_list = list()
step_results_dict = {}
for step in range(1, 24 + 1):
  step_index = y_test_df.index + pd.to_timedelta(step, 'h')
  step_results_df = pd.DataFrame(
      {
          "real": y_test[:, step-1],
          "predictions": y_preds[:, step-1]
      },
      index=step_index
  )
  step_results_df['abs_error'] = abs(step_results_df['real'] - step_results_df['predictions'])
  step_results_df['ape'] = np.where(step_results_df['real'] == 0, np.NaN, 100 * step_results_df['abs_error']/step_results_df['real'])
  step_mape = step_results_df['ape'].mean()
  mape_list.append(step_mape)
  print(f"Step {step} -> MAPE = {step_mape}")

  step_results_dict[step] = step_results_df
mape = np.array(mape_list).mean()
print(f"\nMAPE = {mape}")

<ipython-input-24-98d87e73e4b6>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scaled_df_copy[col + f"_(t-{step_back})"] = scaled_df[col].shift(step_back)
<ipython-input-24-98d87e73e4b6>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scaled_df_copy[col + f"_(t-{step_back})"] = scaled_df[col].shift(step_back)
<ipython-input-24-98d87e73e4b6>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 1/1000 | Loss: 0.2596 | Val loss: 12.7100 - *Checkpoint*
Epoch: 2/1000 | Loss: 0.1080 | Val loss: 10.0835 - *Checkpoint*
Epoch: 3/1000 | Loss: 0.0897 | Val loss: 9.3620 - *Checkpoint*
Epoch: 4/1000 | Loss: 0.0818 | Val loss: 8.5446 - *Checkpoint*
Epoch: 5/1000 | Loss: 0.0753 | Val loss: 8.0250 - *Checkpoint*
Epoch: 6/1000 | Loss: 0.0692 | Val loss: 7.8186 - *Checkpoint*
Epoch: 7/1000 | Loss: 0.0663 | Val loss: 7.2096 - *Checkpoint*
Epoch: 8/1000 | Loss: 0.0657 | Val loss: 8.5980
Epoch: 9/1000 | Loss: 0.0638 | Val loss: 7.0983 - *Checkpoint*
Epoch: 10/1000 | Loss: 0.0631 | Val loss: 8.0411
Epoch: 11/1000 | Loss: 0.0623 | Val loss: 7.1282
Epoch: 12/1000 | Loss: 0.0610 | Val loss: 6.8545 - *Checkpoint*
Epoch: 13/1000 | Loss: 0.0607 | Val loss: 6.9921
Epoch: 14/1000 | Loss: 0.0597 | Val loss: 6.7794 - *Checkpoint*
Epoch: 15/1000 | Loss: 0.0590 | Val loss: 7.3291
Epoch: 16/1000 | Loss: 0.0589 | Val loss: 6.9272
Epoch: 17/1000 | Loss: 0.0585 | Val loss: 6.7417 - *Checkpoint*
Epoch: 18

In [ ]:
step = 24
step_df = step_results_dict[step]
step_df['ape_above_10_flag'] = np.where(step_df['ape'] >= 10., 1, 0)
step_df_grouped = step_df.groupby(by=step_results_dict[1].index.month).sum()
step_df_grouped['ape_above_10_(%)'] = 100 * step_df_grouped['ape_above_10_flag'] / (30 * 24)
step_df_grouped

,real,predictions,abs_error,ape,ape_above_10_flag,ape_above_10_(%)
Timestamp,,,,,,
1,3961348.25,3944251.25,131140.531250,2459.629150,26,3.611111
2,3447525.50,3442842.00,108727.867188,2069.166260,24,3.333333
3,3401897.75,3438634.25,113911.335938,2467.267578,21,2.916667
4,2947588.25,3004475.00,119564.531250,2989.899170,55,7.638889
5,3119189.50,3156895.50,88695.554688,2125.302002,1,0.138889
6,3348490.75,3368263.00,88905.820312,1928.043091,2,0.277778
7,3920082.25,3928837.00,133205.937500,2531.140381,26,3.611111
8,3705299.25,3763367.50,121135.914062,2489.636230,20,2.777778
9,3223079.00,3263685.50,96946.265625,2161.520264,17,2.361111


In [ ]:
step_df_grouped['ape_above_10_(%)'].mean()

3.3217592592592595